# Metrics

Evaluates a trained model accordingly to the metrics specified on the paper

In [ ]:
from google.colab import drive
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


In [ ]:
!pip install tensorflow==2.4.0
!pip install keras==2.4.0
!pip install kymatio
!pip install tqdm
!pip install iterative-stratification

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
ERROR: Could not find a version that satisfies the requirement tensorflow==2.4.0 (from versions: 2.5.0, 2.5.1, 2.5.2, 2.5.3, 2.6.0rc0, 2.6.0rc1, 2.6.0rc2, 2.6.0, 2.6.1, 2.6.2, 2.6.3, 2.6.4, 2.6.5, 2.7.0rc0, 2.7.0rc1, 2.7.0, 2.7.1, 2.7.2, 2.7.3, 2.7.4, 2.8.0rc0, 2.8.0rc1, 2.8.0, 2.8.1, 2.8.2, 2.8.3, 2.8.4, 2.9.0rc0, 2.9.0rc1, 2.9.0rc2, 2.9.0, 2.9.1, 2.9.2, 2.9.3, 2.10.0rc0, 2.10.0rc1, 2.10.0rc2, 2.10.0rc3, 2.10.0, 2.10.1, 2.11.0rc0, 2.11.0rc1, 2.11.0rc2, 2.11.0, 2.11.1, 2.12.0rc0, 2.12.0rc1, 2.12.0)
ERROR: No matching distribution found for tensorflow==2.4.0
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 170.2/170.2 kB 4.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 588.3/588.3 MB 2.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 55.5 MB/s eta 0:00:

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
import sys
sys.path.append("drive/MyDrive/Scattering_Novo/src")

# Scattering Parameters
J=10
Q=2


import numpy as np
from sklearn.metrics import f1_score, precision_score, recall_score, multilabel_confusion_matrix 
from sklearn.model_selection import train_test_split
from ModelHandler import ModelHandler
import pickle
import h5py
from sklearn.metrics import f1_score, precision_score, recall_score     
from tqdm import tqdm
import tensorflow as tf


J=10
Q=2

configs = {
    "N_GRIDS": 5, 
    "SIGNAL_BASE_LENGTH": 12800, 
    "N_CLASS": 26, 
    "USE_NO_LOAD": False, 
    "USE_HAND_AUGMENTATION": False,
    "MARGIN_RATIO": 0.15, 
    "DATASET_PATH": "drive/MyDrive/Scattering_Novo/dataset_original/Synthetic_Full_iHall.hdf5",
    "TRAIN_SIZE": 0.9,
    "FOLDER_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FOLDER_DATA_PATH": "drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/", 
    "FEATURES_FILE_NAME": "features.mat",
    "OUTPUT_CLASSIFICATION_MODELS_PATH": "drive/MyDrive/DeSpaWN-main/classification/without_data_augmentation/",
    "N_EPOCHS_TRAINING": 500,
    "FEAT_NORMALIZATION": False,
    "TRAINING_FLAG": 1,
    "PERCENTUAL": 1.0,
    "INITIAL_EPOCH": 0,
    "TOTAL_MAX_EPOCHS": 5000,
    "SNRdb": None # Nível de ruído em db
}

GRIDSTYPE = "GRIDS3"

folderPath = configs["OUTPUT_CLASSIFICATION_MODELS_PATH"] + "FC_OUTPUT_TYPE_CLASS_" + GRIDSTYPE + "/"
#folderPath = configs["TESTS_FOLDER"] + 'Hybrid_P' + str(int(configs["PERCENTUAL"]*100)) + '_J' + str(J) + '_Q' + str(Q) + '/'
folderDataPath = configs["FOLDER_DATA_PATH"]
signalBaseLength = configs["SIGNAL_BASE_LENGTH"]
ngrids = configs["N_GRIDS"]
trainSize = configs["TRAIN_SIZE"]

dict_data = pickle.load(open(folderDataPath + "data.p", "rb")) # Load data

Using TensorFlow backend.


## Load Features

In [ ]:
import scipy.io as sio


In [ ]:

import os


### Merging the Training Features

In [ ]:

if GRIDSTYPE=="GRIDS3":
  full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids3.mat"
  full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids3.mat"
  training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids3" 
  path_base = "features_train_grids3"
elif GRIDSTYPE=="GRIDS2":
  full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids2.mat"
  full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"
  training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids2" 
  path_base = "features_train_grids2"

  
if not os.path.exists(full_training_set_path):
    #feats= np.zeros([0,14])
    print(full_training_set_path)
    number_of_training_files= 75
    try: 
      for cont in range(number_of_training_files+1): # Number of training files
        path = path_base + str(cont) + ".mat"

        if os.path.isfile(os.path.join(training_features_folder_name,path)):
              csv_path = os.path.join(training_features_folder_name,path)

              print(csv_path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']

                if cont==0:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from training dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(full_training_set_path, {'arr': feats})
    print("Combined and saved!")

    print(feats.shape)

    del feats, imported2


In [ ]:
print("Training data exists? ", os.path.exists(full_training_set_path))
print("Testing data exists? ", os.path.exists(full_testing_set_path))
print(full_testing_set_path)

Training data exists?  True
Testing data exists?  True
drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/combined_features_testing_grids3.mat


In [ ]:
X_train = sio.loadmat(full_training_set_path)
X_train = X_train['arr']
y_train = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_train_.mat")
y_train = y_train['arr']





In [ ]:
print(X_train.shape)
print(y_train.shape)

(7575, 5, 10)
(7575, 5, 26)


### Merging the testing features

In [ ]:
number_of_testing_files= 9
#features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features" 


if GRIDSTYPE=="GRIDS3":
  full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids3.mat"
  full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids3.mat"
  training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids3" 
  testing_features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features_grids3" 
  path_base = "features_train_grids3"

elif GRIDSTYPE=="GRIDS2":
  full_training_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_training_grids2.mat"
  full_testing_set_path = configs["FOLDER_DATA_PATH"] + "combined_features_testing_grids2.mat"
  training_features_folder_name = configs["FOLDER_DATA_PATH"] + "Training_Features_grids2" 
  testing_features_folder_name = configs["FOLDER_DATA_PATH"] + "Testing_Features_grids2" 
  path_base = "features_train_grids2"

if not os.path.exists(full_testing_set_path):
    try: 
      for cont in range(number_of_testing_files): # Number of training files
        path = path_base + str(cont) + ".mat"

        if os.path.isfile(os.path.join(testing_features_folder_name,path)):
              csv_path = os.path.join(testing_features_folder_name,path)
              #print("Extracting from " + path)
              try:
                imported = sio.loadmat(csv_path)
                imported2 = imported['arr']
                #imported2 = imported2.reshape([imported2.shape[1],])

                if cont==0:
                    feats = imported2
                else:
                    #imported2 = imported2.reshape([int(imported2.shape[0]/14),14]) # The shape of a single file
                    feats = np.append(feats,  imported2, axis=0 )
              except:
                print("Fail merging file " + path)
        else:
              print("File is not from testing dataset")
    except:
      print("Training Directory does not exist")

    sio.savemat(configs["FOLDER_PATH"] + "combined_features_testing_grids2" + ".mat", {'arr': feats})
    print("Combined and saved!")
    del feats, imported2
else:
    print("Testing data already merged.")

Testing data already merged.


In [ ]:
os.path.exists(full_training_set_path)

True

In [ ]:
print(full_testing_set_path)

X_test  = sio.loadmat(full_testing_set_path)
X_test = X_test['arr']
y_test  = sio.loadmat(configs["FOLDER_DATA_PATH"] + "yclass_test_.mat")
y_test = y_test['arr']


drive/MyDrive/DeSpaWN-main/extracted_features/without_data_augmentation/Full_Dataset/combined_features_testing_grids3.mat


In [ ]:
print("Shape of Classification Testing Features: ", X_test.shape)
print("Shape of Classification Testing Labels: ", y_test.shape)


Shape of Classification Testing Features:  (841, 5, 10)
Shape of Classification Testing Labels:  (841, 5, 26)


## Features Normalization

In [ ]:
if configs["FEAT_NORMALIZATION"]:
  from sklearn.preprocessing import MultiLabelBinarizer
  from sklearn.preprocessing import MaxAbsScaler 
  for grid in range(5): 
    if grid==0:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = transformer.transform(X_test[:,grid,:])
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = transformer.transform(X_train[:,grid,:])
    else:
      transformer = MaxAbsScaler().fit(X_test[:,grid,:])
      x_test_norm = np.append(x_test_norm, transformer.transform(X_test[:,grid,:]), axis=0 )
      transformer = MaxAbsScaler().fit(X_train[:,grid,:])
      X_train_norm = np.append( X_train_norm, transformer.transform(X_train[:,grid,:]), axis=0)

  x_test_norm = x_test_norm.reshape([-1,5,10])
  X_train_norm = X_train_norm.reshape([-1,5,10])

  print(x_test_norm.shape)
  print(X_train_norm.shape)

  X_test = x_test_norm
  X_train = X_train_norm

  del x_test_norm, X_train_norm

## Choose best performing model

At this point, the model with best performance under the validation set is chosen.

In order to make this choice, the average between f1 macro is verified.

$$
F_1 = \frac{F1_{ON} + F1_{OFF} + F1_{NO EVENT}}{3}
$$

In [ ]:
# Loading Labels
yclass_all = y_train
ytype_all = sio.loadmat(configs["FOLDER_DATA_PATH"] + "ytype_train_.mat")
ytype_all = ytype_all['arr']

print(ytype_all.shape)
print(yclass_all.shape)

(7575, 5, 3)
(7575, 5, 26)


In [ ]:
print(ytype_all.shape)
print(np.argmax(ytype_all, axis=2).shape)
print(np.min(np.argmax(ytype_all, axis=2), axis=1).shape)

(7575, 5, 3)
(7575, 5)
(7575,)


In [ ]:
def choose_model(ytype_all, yclass_all, folderPath):
    from tqdm import tqdm
    from sklearn.preprocessing import MaxAbsScaler
    from sklearn.metrics import f1_score, precision_score, recall_score   
    from PostProcessing import PostProcessing

    #scattering_extract = ModelHandler.loadModel(folderPath + 'scattering_model.h5') # Load scattering model

    threshold = 0.5
    f1_macro, f1_micro = [], []
    for fold in tqdm(range(1, 3)):
        foldFolderPath = folderPath + str(fold) + "/"
        
        train_index = np.load(foldFolderPath + "train_index.npy")
        validation_index = np.load(foldFolderPath + "validation_index.npy")

        bestModel = ModelHandler.loadModel(foldFolderPath + "model_without_detection.h5", type_weights=None) # Load model



        y_train, y_validation = {}, {}
        #y_train["detection"] = ydet_all[train_index]
        #y_validation["detection"] = ydet_all[validation_index]
        y_train["type"] = ytype_all[train_index]
        y_validation["type"] = ytype_all[validation_index]
        y_train["classification"] = yclass_all[train_index]
        y_validation["classification"] = yclass_all[validation_index]

        # Selecting features for classification and type

        x_train_class = X_train[train_index]
        x_validation_class = X_train[validation_index]

        x_train_type = X_train[train_index]
        x_validation_type = X_train[validation_index]

        # Flattening train

        for k in range(x_train_class.shape[0]):
          if k==0:
            B = x_train_class[k].flatten()
            B = B.reshape([1,B.shape[0]])
          else:
            b = x_train_class[k].flatten()
            b = b.reshape([1,b.shape[0]])
            B = np.append(B, b, axis=0)
        x_train_class = B
        del B

        for k in range(x_train_type.shape[0]):
          if k==0:
            B = x_train_type[k].flatten()
            B = B.reshape([1,B.shape[0]])
          else:
            b = x_train_type[k].flatten()
            b = b.reshape([1,b.shape[0]])
            B = np.append(B, b, axis=0)
        x_train_type = B
        del B

        # Flattening validation

        for k in range(x_validation_class.shape[0]):
          if k==0:
            B = x_validation_class[k].flatten()
            B = B.reshape([1,B.shape[0]])
          else:
            b = x_validation_class[k].flatten()
            b = b.reshape([1,b.shape[0]])
            B = np.append(B, b, axis=0)
        x_validation_class = B
        del B

        for k in range(x_validation_type.shape[0]):
          if k==0:
            B = x_validation_type[k].flatten()
            B = B.reshape([1,B.shape[0]])
          else:
            b = x_validation_type[k].flatten()
            b = b.reshape([1,b.shape[0]])
            B = np.append(B, b, axis=0)
        x_validation_type = B
        del B

        #print(x_validation_type.shape)
        #print(x_train_type.shape)
        #print(x_validation_class.shape)
        #print(x_train_class.shape)
        print(y_validation["classification"].shape)
  

        final_prediction = []
        final_groundTruth = []
        for xtype, xclass, yclass, ytype in zip(x_validation_type, x_validation_class, y_validation["classification"], y_validation["type"]):

            pred = bestModel.predict([np.expand_dims(xtype, axis=0),np.expand_dims(xclass, axis=0)])
            prediction = np.max(pred[1][0],axis=0) 
            groundTruth = np.max(yclass,axis=0)

            final_prediction.append(prediction)
            final_groundTruth.append(groundTruth) 

            del xtype, xclass, yclass, ytype

        event_type = np.min(np.argmax(y_validation["type"], axis=2), axis=1)

        final_groundTruth = np.array(final_groundTruth)
        final_prediction = np.array(final_prediction)
    
        f1_macro.append([f1_score(final_groundTruth[event_type == 0] > threshold, final_prediction[event_type == 0] > threshold, average='macro', zero_division=0), 
                         f1_score(final_groundTruth[event_type == 1] > threshold, final_prediction[event_type == 1] > threshold, average='macro', zero_division=0)])
        print(f"Fold {fold}: F1 Macro avg: {np.average(f1_macro[-1]) * 100:.1f}") 

    return np.argmax(np.average(f1_macro, axis=1)) + 1

fold = choose_model(ytype_all, yclass_all, folderPath)


  0%|          | 0/2 [00:00<?, ?it/s]

(733, 5, 26)


 50%|█████     | 1/2 [00:38<00:38, 38.36s/it]

Fold 1: F1 Macro avg: 11.9
(769, 5, 26)


100%|██████████| 2/2 [01:14<00:00, 37.19s/it]

Fold 2: F1 Macro avg: 14.5


## Evaluates the identification

This step generates a dict with the ground truth and the prediction for each test example

In [ ]:
from tqdm import tqdm
from sklearn.preprocessing import MaxAbsScaler

foldFolderPath = folderPath + str(fold) + "/"

train_index = np.load(foldFolderPath + "train_index.npy")
validation_index = np.load(foldFolderPath + "validation_index.npy")

bestModel = ModelHandler.loadModel(foldFolderPath + "model_without_detection.h5", type_weights=None) # Load model



y_train, y_validation = {}, {}
#y_train["detection"] = ydet_all[train_index]
#y_validation["detection"] = ydet_all[validation_index]
y_train["type"] = ytype_all[train_index]
y_validation["type"] = ytype_all[validation_index]
y_train["classification"] = yclass_all[train_index]
y_validation["classification"] = yclass_all[validation_index]

    # Selecting features for classification and type

x_train_class = X_train[train_index]
x_validation_class = X_train[validation_index]

x_train_type = X_train[train_index]
x_validation_type = X_train[validation_index]

x_test_class = X_test
x_test_type = X_test


    # Flattening train

for k in range(x_train_class.shape[0]):
  if k==0:
    B = x_train_class[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_train_class[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_train_class = B
del B

for k in range(x_train_type.shape[0]):
  if k==0:
    B = x_train_type[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_train_type[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_train_type = B
del B

    # Flattening validation

for k in range(x_validation_class.shape[0]):
  if k==0:
    B = x_validation_class[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_validation_class[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_validation_class = B
del B

for k in range(x_validation_type.shape[0]):
  if k==0:
    B = x_validation_type[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_validation_type[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_validation_type = B
del B


    # Flattening Test

for k in range(x_test_class.shape[0]):
  if k==0:
    B = x_test_class[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_test_class[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_test_class = B
del B

for k in range(x_test_type.shape[0]):
  if k==0:
    B = x_test_type[k].flatten()
    B = B.reshape([1,B.shape[0]])
  else:
    b = x_test_type[k].flatten()
    b = b.reshape([1,b.shape[0]])
    B = np.append(B, b, axis=0)
x_test_type = B
del B



final_prediction = []
final_groundTruth = []
for xi, xi_nd, yclass, ytype in zip(x_test_type, x_test_class, dict_data["y_test"]["classification"], dict_data["y_test"]["type"]):
    pred = bestModel.predict([np.expand_dims(xi, axis=0),np.expand_dims(xi_nd, axis=0)])
    prediction = np.max(pred[1][0],axis=0)
    groundTruth = np.max(yclass,axis=0)

    final_prediction.append(prediction)
    final_groundTruth.append(groundTruth) 

    del xi, yclass, ytype

y = {}
y["true"] = final_groundTruth.copy()
y["pred"] = final_prediction.copy()

### F1 Score

#### F1 Macro:
$$
\begin{gather*}
F1_{Macro} = \frac{1}{Y} \sum_{i=1}^{Y} \frac{2 \cdot tp_i}{2 \cdot tp_i + fp_i + fn_i}
\end{gather*}
$$

#### F1 Micro:
$$
\begin{gather*}
F1_{Micro} = \frac{2 \cdot \sum_{i=1}^{Y} tp_i}{\sum_{i=1}^{Y} 2 \cdot tp_i + fp_i + fn_i}
\end{gather*}
$$

- $tp_i$: True positives classifications for appliance $i$
- $fp_i$: False positives classifications for appliance $i$
- $fn_i$: False negatives classifications for appliance $i$

In [ ]:
from sklearn.metrics import f1_score

threshold = 0.5
f1_macro = f1_score(np.array(y["true"]) > threshold, np.array(y["pred"]) > threshold, average='macro')
f1_micro = f1_score(np.array(y["true"]) > threshold, np.array(y["pred"]) > threshold, average='micro')

print(f"Fold {fold} - F1 Macro: {f1_macro * 100:.1f}, F1 Micro: {f1_micro * 100:.1f}")

Fold 2 - F1 Macro: 13.8, F1 Micro: 19.5


### Accuracy (ACC)

$$
\begin{gather*}
ACC_i = \frac{CCE_i}{TNE_i} \\ \\
ACC = \frac{1}{Y} \sum_{i = 1}^{Y} ACC_i
\end{gather*}
$$

- $ACC_i$: Accuracy for appliance $i$
- $CCE_i$: Load connected successfully identified
- $TNE_i$: Total of connected events

In [ ]:
threshold = 0.5

correct_on = np.zeros((26,1))
total_on = np.zeros((26,1))
correct_off = np.zeros((26,1))
total_off = np.zeros((26,1))
correct_no_event = np.zeros((26,1))
total_no_event = np.zeros((26,1))

for ytype, ytrue, ypred in zip(dict_data["y_test"]["type"], y["true"], y["pred"]):
    event_type = np.min(np.argmax(ytype, axis=1))
    if event_type == 0:
        correct_on[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_on[ytrue > threshold] += 1
    elif event_type == 1:
        correct_off[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_off[ytrue > threshold] += 1
    else:
        correct_no_event[np.bitwise_and(ytrue > threshold, ypred > threshold)] += 1
        total_no_event[ytrue > threshold] += 1

acc_on = 100 * np.average(np.nan_to_num(correct_on/total_on))
acc_off = 100 * np.average(np.nan_to_num(correct_off/total_off))
acc_no_event = 100 * np.average(np.nan_to_num(correct_no_event/total_no_event))
acc_total = 100 * np.average(np.nan_to_num((correct_on + correct_off + correct_no_event)/(total_on + total_off + total_no_event)))

print(f"Fold {fold} - Acc on: {acc_on:.1f}, Acc off: {acc_off:.1f}, Acc no event: {acc_no_event:.1f} Acc total: {acc_total:.1f}")

Fold 2 - Acc on: 9.6, Acc off: 10.9, Acc no event: 0.0 Acc total: 10.7


<ipython-input-24-6293edaa4ae2>:23: RuntimeWarning: invalid value encountered in true_divide
  acc_off = 100 * np.average(np.nan_to_num(correct_off/total_off))
<ipython-input-24-6293edaa4ae2>:24: RuntimeWarning: invalid value encountered in true_divide
  acc_no_event = 100 * np.average(np.nan_to_num(correct_no_event/total_no_event))


## Detection Metrics

### D
$$
\begin{gather*}
D = \frac{ \sum_{i=1}^{A} |d(i) - ev(i)|}{A}
\end{gather*}
$$

- `A`: Total of events correctly detected ($\pm$ 10 semi cycles tolerance)
- `d(i)`: Detection for appliance $i$
- `ev(i)`: Ground truth detection for appliance $i$

## PC

$$
\begin{gather*}
PC = \frac{A}{N}
\end{gather*}
$$

- `A`: Total of events correctly detected ($\pm$ 10 semi cycles tolerance)
- `N`: Total of events

In [ ]:
from PostProcessing import PostProcessing
from DataHandler import DataHandler

postProcessing = PostProcessing(configs=configs)
dataHandler = DataHandler(configs=configs)

general_qtd_test = dict_data["y_test"]["group"]

foldFolderPath = folderPath + str(fold) + "/"

train_index = np.load(foldFolderPath + "train_index.npy")



print(f"-------------- FOLD {fold} ---------------")
pcMetric = postProcessing.checkModel2(bestModel, x_test_type, x_test_class, dict_data["y_test"], general_qtd=general_qtd_test, print_error=False)

-------------- FOLD 2 ---------------
Total time: 38.49111354699858, Average Time: 0.04576826818905895
LIT-SYN-1 PCmetric: (0.8222222222222222, 0.9210526315789473, 0.8674698795180723)
LIT-SYN-2 PCmetric: (0.7956204379562044, 0.855072463768116, 0.8254545454545454)
LIT-SYN-3 PCmetric: (0.6174496644295302, 0.710691823899371, 0.6655844155844156)
LIT-SYN-8 PCmetric: (0.47674418604651164, 0.5168539325842697, 0.49714285714285716)
LIT-SYN-All PCmetric: (0.6690647482014388, 0.7358490566037735, 0.7027348394768134)


In [ ]:
pcMetric

pc_on = pcMetric[4][0]
pc_off = pcMetric[4][1]
pc_all = pcMetric[4][2]

In [ ]:
# Saving the Results

import tables
import numpy as np

row = [acc_on*0.01, acc_off*0.01, acc_total*0.01, f1_macro, f1_micro, pc_on, pc_off, pc_all]

print(np.array(row))




[0.09596738 0.10944394 0.10660039 0.13820568 0.19493345 0.66906475
 0.73584906 0.70273484]


In [ ]:
fold

2